Tool‑calling Agents — LLM decides which tool/API to call

When to use: You need dynamic decisions (“should I call SQL? a REST API? run a UC function?”) without orchestrating every step in code.

In [0]:
%pip install unitycatalog-ai[databricks] databricks-langchain langchain


In [0]:
dbutils.library.restartPython()

In [0]:
# 1) Make sure the notebook uses its built-in Spark session (not Spark Connect)
import os
for k in ("DATABRICKS_HOST", "DATABRICKS_TOKEN", "DATABRICKS_CONFIG_PROFILE"):
    os.environ.pop(k, None)

# 2) (Optional) Explicitly bind to the default notebook Spark session
from databricks.connect import DatabricksSession
spark = DatabricksSession.builder.getOrCreate()  # no parameters!

# 3) Sanity check
spark.sql("SELECT 1").show()


In [0]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient
from databricks_langchain import UCFunctionToolkit, ChatDatabricks, set_uc_function_client
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate

# IMPORTANT: local mode avoids serverless + Spark Connect in notebooks
client = DatabricksFunctionClient(execution_mode="local")
set_uc_function_client(client)   # ensure tool calls use the same local-mode client

CATALOG, SCHEMA = "hive_metastore", "default"

def add_numbers(number_1: float, number_2: float) -> float:
     """Add two numbers and return the sum."""
     return number_1 + number_2

# create the UC function using the same client
client.create_python_function(func=add_numbers, catalog=CATALOG, schema=SCHEMA, replace=True)
func_name = f"{CATALOG}.{SCHEMA}.add_numbers"

toolkit = UCFunctionToolkit(function_names=[func_name])
llm = ChatDatabricks(model="databricks-meta-llama-3-3-70b-instruct", temperature=0)  # 'model' is also accepted

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful. Use tools when needed, otherwise answer directly."),
    ("human", "{input}")
])

agent = create_tool_calling_agent(llm, toolkit.tools, prompt)
executor = AgentExecutor(agent=agent, tools=toolkit.tools, verbose=True)

print(executor.invoke({"input": "What is 36939.0 + 8922.4?"}))